# Apache Spark Docker Cluster Demo

**Professional demonstration of Apache Spark running on Docker with comprehensive event logging**

---

## 🎯 Overview

This notebook demonstrates a complete Apache Spark setup with:

- **Docker Cluster Mode**: Spark master and workers running in containers
- **Event Logging**: Comprehensive tracking of all Spark operations
- **Advanced Analytics**: Real-world data processing examples
- **Monitoring Integration**: Web UIs for cluster and application monitoring

---

## 🏗️ Architecture

### Cluster Components
- **Spark Master**: Cluster coordinator (port 7077, UI port 8080)
- **Spark Workers**: Execution nodes 
- **History Server**: Event log analysis (port 18080)
- **JupyterLab**: Alternative notebook interface (port 8888)

### Driver Configuration
- **External Driver**: VS Code notebook connects to Docker cluster
- **Event Storage**: Local filesystem shared with containers
- **Network**: `host.docker.internal` for cross-container communication

---

## 📊 Monitoring Dashboard URLs

| Service | URL | Description |
|---------|-----|-------------|
| Cluster Master | http://localhost:8080 | Live cluster status |
| Application UI | http://host.docker.internal:4040 | Current job details |
| History Server | http://localhost:18080 | Historical event logs |
| JupyterLab | http://localhost:8888 | Container notebook interface |

---

## 🚀 Quick Start

1. **Start containers**: `docker compose up -d`
2. **Run cells below**: Execute the Spark configuration and analytics
3. **Monitor execution**: Use the dashboard URLs above
4. **View event logs**: Check History Server for detailed execution logs

---

## ? Technical Specifications

- **Spark Version**: 3.5.0
- **Event Compression**: zstd algorithm
- **Serialization**: Kryo for performance
- **Memory**: 1GB driver + 1GB executor
- **Adaptive Query Execution**: Enabled
- **Event Directory**: `./events/` (shared volume)

In [6]:
# =============================================================================
# SPARK CLUSTER CONFIGURATION
# =============================================================================

import os
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

print("🐳 Initializing Spark Docker Cluster Connection")
print("=" * 60)

# Configuration
current_dir = os.getcwd()
workspace_root = os.path.dirname(current_dir)
events_dir = os.path.join(workspace_root, "events")
events_uri = f"file://{events_dir}"

# Ensure events directory exists
os.makedirs(events_dir, exist_ok=True)

print(f"📁 Events Directory: {events_dir}")
print(f"⏰ Session Start: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

🐳 Initializing Spark Docker Cluster Connection
📁 Events Directory: /Users/congdinh/Downloads/work/content/de/spark-docker/events
⏰ Session Start: 2025-09-03 21:41:51


In [7]:
# =============================================================================
# SPARK SESSION CREATION
# =============================================================================

print("🚀 Creating Spark Session...")

# Build Spark session with optimized configuration
spark = SparkSession.builder \
    .appName("DockerClusterDemo") \
    .master("spark://localhost:7077") \
    .config("spark.driver.host", "host.docker.internal") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.driver.port", "7078") \
    .config("spark.blockManager.port", "7079") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", events_uri) \
    .config("spark.eventLog.compress", "true") \
    .getOrCreate()

# Display session information
print("✅ Spark Session Created Successfully!")
print(f"📋 Application ID: {spark.sparkContext.applicationId}")
print(f"🎯 Master: {spark.sparkContext.master}")
print(f"🌐 Application UI: {spark.sparkContext.uiWebUrl}")
print(f"📝 Event Logging: {spark.sparkContext.getConf().get('spark.eventLog.enabled')}")
print(f"📁 Event Directory: {events_uri}")

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

🚀 Creating Spark Session...
✅ Spark Session Created Successfully!
📋 Application ID: app-20250903144158-0016
🎯 Master: spark://localhost:7077
🌐 Application UI: http://host.docker.internal:4040
📝 Event Logging: true
📁 Event Directory: file:///Users/congdinh/Downloads/work/content/de/spark-docker/events


25/09/03 21:41:58 WARN Utils: Service 'sparkDriver' could not bind on port 7078. Attempting port 7079.
25/09/03 21:41:58 WARN Utils: Service 'sparkDriver' could not bind on port 7079. Attempting port 7080.
25/09/03 21:41:58 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 7079. Attempting port 7080.
25/09/03 21:41:58 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 7080. Attempting port 7081.


25/09/03 21:42:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
# =============================================================================
# DATA PREPARATION
# =============================================================================

print("📊 Creating Sample Dataset...")

# Sample employee data
employee_data = [
    ("Alice Johnson", 28, "Engineering", 75000, "Senior"),
    ("Bob Smith", 35, "Sales", 65000, "Manager"),
    ("Catherine Lee", 32, "Marketing", 70000, "Senior"),
    ("David Brown", 29, "Engineering", 80000, "Senior"),
    ("Eva Wilson", 31, "Sales", 68000, "Senior"),
    ("Frank Miller", 26, "Engineering", 72000, "Junior"),
    ("Grace Davis", 27, "Marketing", 69000, "Junior"),
    ("Henry Taylor", 34, "Sales", 71000, "Manager"),
    ("Ivy Chen", 30, "Engineering", 85000, "Senior"),
    ("Jack Anderson", 25, "Marketing", 63000, "Junior")
]

# Define schema columns
columns = ["Name", "Age", "Department", "Salary", "Level"]

# Create DataFrame
df = spark.createDataFrame(employee_data, columns)

# Cache the DataFrame for performance
df.cache()

print(f"✅ Dataset Created: {df.count()} employees")
print("📋 Sample Data:")
df.show(10, truncate=False)

📊 Creating Sample Dataset...


✅ Dataset Created: 10 employees
📋 Sample Data:
+-------------+---+-----------+------+-------+
|Name         |Age|Department |Salary|Level  |
+-------------+---+-----------+------+-------+
|Alice Johnson|28 |Engineering|75000 |Senior |
|Bob Smith    |35 |Sales      |65000 |Manager|
|Catherine Lee|32 |Marketing  |70000 |Senior |
|David Brown  |29 |Engineering|80000 |Senior |
|Eva Wilson   |31 |Sales      |68000 |Senior |
|Frank Miller |26 |Engineering|72000 |Junior |
|Grace Davis  |27 |Marketing  |69000 |Junior |
|Henry Taylor |34 |Sales      |71000 |Manager|
|Ivy Chen     |30 |Engineering|85000 |Senior |
|Jack Anderson|25 |Marketing  |63000 |Junior |
+-------------+---+-----------+------+-------+

+-------------+---+-----------+------+-------+
|Name         |Age|Department |Salary|Level  |
+-------------+---+-----------+------+-------+
|Alice Johnson|28 |Engineering|75000 |Senior |
|Bob Smith    |35 |Sales      |65000 |Manager|
|Catherine Lee|32 |Marketing  |70000 |Senior |
|David Brown

In [9]:
# =============================================================================
# DEPARTMENT ANALYSIS
# =============================================================================

print("🏢 Analyzing Department Statistics...")

# Group by department and calculate metrics
dept_stats = df.groupBy("Department") \
    .agg(
        F.count("*").alias("Employee_Count"),
        F.avg("Age").alias("Avg_Age"),
        F.avg("Salary").alias("Avg_Salary"),
        F.min("Salary").alias("Min_Salary"),
        F.max("Salary").alias("Max_Salary")
    ) \
    .orderBy(F.desc("Employee_Count"))

print("📊 Department Statistics:")
dept_stats.show()

# Level distribution analysis
level_stats = df.groupBy("Department", "Level") \
    .count() \
    .orderBy("Department", F.desc("count"))

print("👥 Level Distribution by Department:")
level_stats.show()

🏢 Analyzing Department Statistics...
📊 Department Statistics:
+-----------+--------------+------------------+-----------------+----------+----------+
| Department|Employee_Count|           Avg_Age|       Avg_Salary|Min_Salary|Max_Salary|
+-----------+--------------+------------------+-----------------+----------+----------+
|Engineering|             4|             28.25|          78000.0|     72000|     85000|
|      Sales|             3|33.333333333333336|          68000.0|     65000|     71000|
|  Marketing|             3|              28.0|67333.33333333333|     63000|     70000|
+-----------+--------------+------------------+-----------------+----------+----------+

👥 Level Distribution by Department:
+-----------+--------------+------------------+-----------------+----------+----------+
| Department|Employee_Count|           Avg_Age|       Avg_Salary|Min_Salary|Max_Salary|
+-----------+--------------+------------------+-----------------+----------+----------+
|Engineering|        

In [10]:
# =============================================================================
# SALARY ANALYSIS
# =============================================================================

print("💰 Performing Salary Analysis...")

# High earners analysis
high_earners = df.filter(F.col("Salary") > 70000) \
    .orderBy(F.desc("Salary"))

print("🌟 High Earners (Salary > $70,000):")
high_earners.show()

# Salary comparison with department average
dept_avg_salary = dept_stats.select("Department", "Avg_Salary")

# Join employee data with department averages
salary_comparison = df.join(dept_avg_salary, "Department", "left") \
    .withColumn("Salary_vs_Avg", F.round(F.col("Salary") - F.col("Avg_Salary"), 2)) \
    .withColumn("Performance_Category", 
        F.when(F.col("Salary_vs_Avg") > 5000, "Above Average")
         .when(F.col("Salary_vs_Avg") < -5000, "Below Average")
         .otherwise("Average")
    ) \
    .select("Name", "Department", "Salary", "Avg_Salary", "Salary_vs_Avg", "Performance_Category")

print("📈 Salary vs Department Average:")
salary_comparison.orderBy(F.desc("Salary_vs_Avg")).show(truncate=False)

💰 Performing Salary Analysis...
🌟 High Earners (Salary > $70,000):
+-------------+---+-----------+------+-------+
|         Name|Age| Department|Salary|  Level|
+-------------+---+-----------+------+-------+
|     Ivy Chen| 30|Engineering| 85000| Senior|
|  David Brown| 29|Engineering| 80000| Senior|
|Alice Johnson| 28|Engineering| 75000| Senior|
| Frank Miller| 26|Engineering| 72000| Junior|
| Henry Taylor| 34|      Sales| 71000|Manager|
+-------------+---+-----------+------+-------+

📈 Salary vs Department Average:
+-------------+-----------+------+-----------------+-------------+--------------------+
|Name         |Department |Salary|Avg_Salary       |Salary_vs_Avg|Performance_Category|
+-------------+-----------+------+-----------------+-------------+--------------------+
|Ivy Chen     |Engineering|85000 |78000.0          |7000.0       |Above Average       |
|Henry Taylor |Sales      |71000 |68000.0          |3000.0       |Average             |
|Catherine Lee|Marketing  |70000 |673

In [11]:
# =============================================================================
# AGE GROUP ANALYSIS
# =============================================================================

print("👥 Analyzing Age Demographics...")

# Create age groups
df_with_age_groups = df.withColumn("Age_Group",
    F.when(F.col("Age") <= 27, "Young (≤27)")
     .when(F.col("Age") <= 32, "Mid-Career (28-32)")
     .otherwise("Experienced (33+)")
)

# Age group distribution
age_group_distribution = df_with_age_groups.groupBy("Age_Group") \
    .agg(
        F.count("*").alias("Count"),
        F.avg("Salary").alias("Avg_Salary"),
        F.avg("Age").alias("Avg_Age")
    ) \
    .orderBy("Avg_Age")

print("🎂 Age Group Distribution:")
age_group_distribution.show()

# Cross-analysis: Age Group vs Department
age_dept_analysis = df_with_age_groups.groupBy("Department", "Age_Group") \
    .count() \
    .orderBy("Department", "Age_Group")

print("🏢 Age Groups by Department:")
age_dept_analysis.show()

👥 Analyzing Age Demographics...
🎂 Age Group Distribution:
+------------------+-----+----------+-------+
|         Age_Group|Count|Avg_Salary|Avg_Age|
+------------------+-----+----------+-------+
|       Young (≤27)|    3|   68000.0|   26.0|
|Mid-Career (28-32)|    5|   75600.0|   30.0|
| Experienced (33+)|    2|   68000.0|   34.5|
+------------------+-----+----------+-------+

🏢 Age Groups by Department:
+------------------+-----+----------+-------+
|         Age_Group|Count|Avg_Salary|Avg_Age|
+------------------+-----+----------+-------+
|       Young (≤27)|    3|   68000.0|   26.0|
|Mid-Career (28-32)|    5|   75600.0|   30.0|
| Experienced (33+)|    2|   68000.0|   34.5|
+------------------+-----+----------+-------+

🏢 Age Groups by Department:
+-----------+------------------+-----+
| Department|         Age_Group|count|
+-----------+------------------+-----+
|Engineering|Mid-Career (28-32)|    3|
|Engineering|       Young (≤27)|    1|
|  Marketing|Mid-Career (28-32)|    1|
|  Mar

In [12]:
# =============================================================================
# SUMMARY & CLEANUP
# =============================================================================

print("📊 Final Summary Report")
print("=" * 60)

# Calculate key metrics
total_employees = df.count()
total_departments = df.select("Department").distinct().count()
avg_salary = df.agg(F.avg("Salary")).collect()[0][0]
avg_age = df.agg(F.avg("Age")).collect()[0][0]
high_earner_count = high_earners.count()

print(f"👥 Total Employees: {total_employees}")
print(f"🏢 Total Departments: {total_departments}")
print(f"💰 Average Salary: ${avg_salary:,.2f}")
print(f"🎂 Average Age: {avg_age:.1f} years")
print(f"🌟 High Earners (>$70K): {high_earner_count}")

# Performance metrics
print(f"\n⚡ Performance Metrics:")
print(f"📝 Event Logging: {'✅ Enabled' if spark.sparkContext.getConf().get('spark.eventLog.enabled') == 'true' else '❌ Disabled'}")
print(f"💾 DataFrame Cached: {'✅ Yes' if df.is_cached else '❌ No'}")
print(f"🔗 Cluster Mode: {'✅ Docker Cluster' if 'spark://' in spark.sparkContext.master else '❌ Local Mode'}")

# Unpersist cached DataFrame
df.unpersist()
print(f"\n🗑️ Cache cleared")

print("=" * 60)
print(f"⏰ Analysis completed at: {time.strftime('%Y-%m-%d %H:%M:%S')}")

📊 Final Summary Report
👥 Total Employees: 10
🏢 Total Departments: 3
💰 Average Salary: $71,800.00
🎂 Average Age: 29.7 years
🌟 High Earners (>$70K): 5

⚡ Performance Metrics:
📝 Event Logging: ✅ Enabled
💾 DataFrame Cached: ✅ Yes
🔗 Cluster Mode: ✅ Docker Cluster

🗑️ Cache cleared
⏰ Analysis completed at: 2025-09-03 21:42:48
👥 Total Employees: 10
🏢 Total Departments: 3
💰 Average Salary: $71,800.00
🎂 Average Age: 29.7 years
🌟 High Earners (>$70K): 5

⚡ Performance Metrics:
📝 Event Logging: ✅ Enabled
💾 DataFrame Cached: ✅ Yes
🔗 Cluster Mode: ✅ Docker Cluster

🗑️ Cache cleared
⏰ Analysis completed at: 2025-09-03 21:42:48


In [13]:
# =============================================================================
# SESSION TERMINATION & EVENT LOG VERIFICATION
# =============================================================================

print("🛑 Terminating Spark Session...")

# Get session info before stopping
app_id = spark.sparkContext.applicationId
app_name = spark.sparkContext.appName

# Stop the Spark session
spark.stop()

print(f"✅ Spark session terminated successfully")
print(f"📋 Application: {app_name}")
print(f"🆔 Application ID: {app_id}")

# Verify event logs were created
import glob
event_files = glob.glob(os.path.join(events_dir, "*"))
print(f"\n📁 Event Log Files ({len(event_files)} total):")

for event_file in sorted(event_files):
    file_name = os.path.basename(event_file)
    file_size = os.path.getsize(event_file)
    print(f"   📄 {file_name} ({file_size:,} bytes)")

print(f"\n🔍 Monitor URLs (may require application restart):")
print(f"   📊 Cluster Master: http://localhost:8080")
print(f"   📈 History Server: http://localhost:18080")
print(f"   🐳 JupyterLab: http://localhost:8888")

print("\n" + "=" * 60)
print("🎉 Docker Cluster Demo Completed Successfully!")
print("=" * 60)

🛑 Terminating Spark Session...
✅ Spark session terminated successfully
📋 Application: DockerClusterDemo
🆔 Application ID: app-20250903144158-0016

📁 Event Log Files (5 total):
   📄 app-20250903143443-0014.zstd (223,506 bytes)
   📄 app-20250903143812-0015.zstd (224,462 bytes)
   📄 app-20250903144158-0016.zstd (278,983 bytes)
   📄 local-1756909626239 (189,404 bytes)
   📄 local-1756909983686 (1,001,424 bytes)

🔍 Monitor URLs (may require application restart):
   📊 Cluster Master: http://localhost:8080
   📈 History Server: http://localhost:18080
   🐳 JupyterLab: http://localhost:8888

🎉 Docker Cluster Demo Completed Successfully!


---

## 🎯 What We Accomplished

This notebook successfully demonstrated:

### ✅ **Technical Achievements**
- **Docker Cluster Integration**: External driver connecting to containerized Spark cluster
- **Event Logging**: Comprehensive tracking with compressed storage
- **Advanced Analytics**: Multi-dimensional data analysis with joins and aggregations
- **Performance Optimization**: DataFrame caching and adaptive query execution

### 📊 **Analytics Results**
- **Employee Dataset**: 10 records across 3 departments
- **Department Analysis**: Engineering, Sales, Marketing statistics
- **Salary Analysis**: Performance categorization and comparisons
- **Demographics**: Age group distributions and cross-analysis

### 🔧 **Infrastructure Components**
- **Spark Master**: Cluster coordination and resource management
- **Spark Workers**: Distributed computation execution
- **History Server**: Event log analysis and visualization
- **Event Storage**: Persistent logging with compression

---

## 📚 Next Steps

1. **Scale the Dataset**: Replace sample data with real datasets
2. **Advanced Analytics**: Implement machine learning pipelines
3. **Performance Tuning**: Optimize cluster resources for larger workloads
4. **Monitoring**: Set up alerting and metrics collection

---

## 📞 Resources

- **Apache Spark Documentation**: https://spark.apache.org/docs/latest/
- **Docker Compose Reference**: https://docs.docker.com/compose/
- **PySpark API**: https://spark.apache.org/docs/latest/api/python/

---

*Demo completed successfully! All events have been logged and are available in the History Server.*